In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datathon24playground/datathon-2024-playground/sample_submission.csv
/kaggle/input/datathon24playground/datathon-2024-playground/ip_address_mapping.csv
/kaggle/input/datathon24playground/datathon-2024-playground/train.csv
/kaggle/input/datathon24playground/datathon-2024-playground/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/datathon24playground/datathon-2024-playground/train.csv")
test = pd.read_csv("/kaggle/input/datathon24playground/datathon-2024-playground/test.csv")
train.head()

,id_pengguna,waktu_pendaftaran_akun,waktu_pembelian,total_harga_pembelian,id_perangkat,sumber,browser,gender,umur,alamat_IP,fraud
0,286873,6/7/2015 12:58,9/27/2015 16:32,14,DNTIEECGGDXHF,Iklan,Opera,F,38,2.417207e+08,0
1,244392,5/1/2015 15:29,8/27/2015 14:21,11,MECRFNJGDPGOB,SEO,Chrome,M,36,4.172794e+09,0
2,20720,3/29/2015 18:04,6/13/2015 6:50,52,WLTSYVLLASJOZ,SEO,Internet Explorer,M,26,1.113433e+09,0
3,198755,1/29/2015 0:49,2/17/2015 21:49,13,TWGNWOWURZFMY,SEO,Safari,F,38,1.339309e+09,0
4,64996,1/17/2015 19:30,3/1/2015 9:17,23,XMQENRCEILYAJ,Langsung,Safari,F,19,4.602382e+08,0


In [3]:
df_ip = pd.read_csv("/kaggle/input/datathon24playground/datathon-2024-playground/ip_address_mapping.csv")
df_ip.head()

,batas_bawah_alamat_IP,batas_atas_alamat_IP,negara
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China


In [4]:
train["waktu_pendaftaran_akun"] = pd.to_datetime(train["waktu_pendaftaran_akun"])
train["waktu_pembelian"] = pd.to_datetime(train["waktu_pembelian"])
train["alamat_IP"] = train["alamat_IP"].astype(int)

test["waktu_pendaftaran_akun"] = pd.to_datetime(test["waktu_pendaftaran_akun"])
test["waktu_pembelian"] = pd.to_datetime(test["waktu_pembelian"])
test["alamat_IP"] = test["alamat_IP"].astype(int)


In [5]:
bins = pd.concat([pd.Series(16777216), df_ip["batas_atas_alamat_IP"]])
train["country"] = pd.cut(train["alamat_IP"], bins=bins, labels=df_ip["negara"], ordered = False)

test["country"] = pd.cut(test["alamat_IP"], bins=bins, labels=df_ip["negara"], ordered = False)

In [6]:
train.head()

,id_pengguna,waktu_pendaftaran_akun,waktu_pembelian,total_harga_pembelian,id_perangkat,sumber,browser,gender,umur,alamat_IP,fraud,country
0,286873,2015-06-07 12:58:00,2015-09-27 16:32:00,14,DNTIEECGGDXHF,Iklan,Opera,F,38,241720722,0,China
1,244392,2015-05-01 15:29:00,2015-08-27 14:21:00,11,MECRFNJGDPGOB,SEO,Chrome,M,36,4172794236,0,NaN
2,20720,2015-03-29 18:04:00,2015-06-13 06:50:00,52,WLTSYVLLASJOZ,SEO,Internet Explorer,M,26,1113432879,0,United States
3,198755,2015-01-29 00:49:00,2015-02-17 21:49:00,13,TWGNWOWURZFMY,SEO,Safari,F,38,1339308896,0,Germany
4,64996,2015-01-17 19:30:00,2015-03-01 09:17:00,23,XMQENRCEILYAJ,Langsung,Safari,F,19,460238214,0,Philippines


In [7]:
train["country"] = train["country"].astype("object")
train["country"] = train["country"].fillna("Unknown")

test["country"] = test["country"].astype("object")
test["country"] = test["country"].fillna("Unknown")

In [8]:
train["day_difference"] = (train["waktu_pembelian"] - train["waktu_pendaftaran_akun"]).dt.days

test["day_difference"] = (test["waktu_pembelian"] - test["waktu_pendaftaran_akun"]).dt.days

In [9]:
train["id_perangkat"].nunique()

71299

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
one_hot_encoder = OneHotEncoder(sparse=False)
train_cat = train[["sumber","browser", "gender"]]
cat_encoded_arr = one_hot_encoder.fit_transform(train_cat)
cat_encoded_df = pd.DataFrame(cat_encoded_arr, columns=one_hot_encoder.get_feature_names_out(train_cat.columns))

test_cat = test[["sumber","browser", "gender"]]
test_cat_encoded_arr = one_hot_encoder.fit_transform(test_cat)
test_cat_encoded_df = pd.DataFrame(test_cat_encoded_arr, columns=one_hot_encoder.get_feature_names_out(test_cat.columns))

cat_encoded_df.head()

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,sumber_Iklan,sumber_Langsung,sumber_SEO,browser_Chrome,browser_Firefox,browser_Internet Explorer,browser_Opera,browser_Safari,gender_F,gender_M
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [12]:
cat_encoded_df["fraud"] = train["fraud"]
cat_encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75556 entries, 0 to 75555
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sumber_Iklan               75556 non-null  float64
 1   sumber_Langsung            75556 non-null  float64
 2   sumber_SEO                 75556 non-null  float64
 3   browser_Chrome             75556 non-null  float64
 4   browser_Firefox            75556 non-null  float64
 5   browser_Internet Explorer  75556 non-null  float64
 6   browser_Opera              75556 non-null  float64
 7   browser_Safari             75556 non-null  float64
 8   gender_F                   75556 non-null  float64
 9   gender_M                   75556 non-null  float64
 10  fraud                      75556 non-null  int64  
dtypes: float64(10), int64(1)
memory usage: 6.3 MB


In [13]:
cat_corr_matrix = cat_encoded_df.corr()
cat_corr_matrix["fraud"].sort_values(ascending=False)

fraud                        1.000000
sumber_Langsung              0.015368
browser_Chrome               0.013926
gender_M                     0.005577
browser_Firefox             -0.000298
sumber_Iklan                -0.002241
browser_Safari              -0.003808
gender_F                    -0.005577
browser_Opera               -0.006729
browser_Internet Explorer   -0.010013
sumber_SEO                  -0.010370
Name: fraud, dtype: float64

In [14]:
cat_encoded_df_useful = cat_encoded_df[["sumber_Langsung", "browser_Chrome", "browser_Internet Explorer", "sumber_SEO"]]
cat_encoded_df_useful["sumber_Langsung_or_SEO"] = (cat_encoded_df_useful["sumber_Langsung"].astype("bool")  | cat_encoded_df_useful["sumber_SEO"].astype("bool")).astype("int")
cat_encoded_df_useful["browser_c_or_ie"] = (cat_encoded_df_useful["browser_Chrome"].astype("bool")  | cat_encoded_df_useful["browser_Internet Explorer"].astype("bool")).astype("int")

test_cat_encoded_df_useful = test_cat_encoded_df[["sumber_Langsung", "browser_Chrome", "browser_Internet Explorer", "sumber_SEO"]]
test_cat_encoded_df_useful["sumber_Langsung_or_SEO"] = (test_cat_encoded_df_useful["sumber_Langsung"].astype("bool")  | test_cat_encoded_df_useful["sumber_SEO"].astype("bool")).astype("int")
test_cat_encoded_df_useful["browser_c_or_ie"] = (test_cat_encoded_df_useful["browser_Chrome"].astype("bool")  | test_cat_encoded_df_useful["browser_Internet Explorer"].astype("bool")).astype("int")


cat_encoded_final = cat_encoded_df_useful[["sumber_Langsung_or_SEO", "browser_c_or_ie"]]
test_cat_encoded_final = test_cat_encoded_df_useful[["sumber_Langsung_or_SEO", "browser_c_or_ie"]]

/tmp/ipykernel_33/1843802615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_encoded_df_useful["sumber_Langsung_or_SEO"] = (cat_encoded_df_useful["sumber_Langsung"].astype("bool")  | cat_encoded_df_useful["sumber_SEO"].astype("bool")).astype("int")
/tmp/ipykernel_33/1843802615.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_encoded_df_useful["browser_c_or_ie"] = (cat_encoded_df_useful["browser_Chrome"].astype("bool")  | cat_encoded_df_useful["browser_Internet Explorer"].astype("bool")).astyp

In [15]:
train = pd.concat([train, cat_encoded_final], axis=1)
test = pd.concat([test, test_cat_encoded_final], axis=1)

train.head()

,id_pengguna,waktu_pendaftaran_akun,waktu_pembelian,total_harga_pembelian,id_perangkat,sumber,browser,gender,umur,alamat_IP,fraud,country,day_difference,sumber_Langsung_or_SEO,browser_c_or_ie
0,286873,2015-06-07 12:58:00,2015-09-27 16:32:00,14,DNTIEECGGDXHF,Iklan,Opera,F,38,241720722,0,China,112,0,0
1,244392,2015-05-01 15:29:00,2015-08-27 14:21:00,11,MECRFNJGDPGOB,SEO,Chrome,M,36,4172794236,0,Unknown,117,1,1
2,20720,2015-03-29 18:04:00,2015-06-13 06:50:00,52,WLTSYVLLASJOZ,SEO,Internet Explorer,M,26,1113432879,0,United States,75,1,1
3,198755,2015-01-29 00:49:00,2015-02-17 21:49:00,13,TWGNWOWURZFMY,SEO,Safari,F,38,1339308896,0,Germany,19,1,0
4,64996,2015-01-17 19:30:00,2015-03-01 09:17:00,23,XMQENRCEILYAJ,Langsung,Safari,F,19,460238214,0,Philippines,42,1,0


In [16]:
train.groupby("id_perangkat")["fraud"].sum()

id_perangkat
AAAXXOZJRZRAO    2
AABFGRPBQHWFQ    0
AABJEESQPJHDQ    0
AABNLBQBKMWRG    0
AABOMYDURXEQT    0
                ..
ZZYXERMSKABCQ    0
ZZZCCVFIRNYDB    0
ZZZIKLJSVSQMF    0
ZZZKJIZHJEDFN    0
ZZZMVOGBAJVTM    0
Name: fraud, Length: 71299, dtype: int64

In [17]:
train["freq_id_perangkat"] = train["id_perangkat"].map(train["id_perangkat"].value_counts())
train["kategori_id_perangkat"] = np.where(train['freq_id_perangkat'] > 1, '>1', 'once')

train["freq_country"] = train["country"].map(train["country"].value_counts())
train["kategori_country"] = np.where(train['freq_country'] > 1000, '>1000', '<=1000')
train.head()


test["freq_id_perangkat"] = test["id_perangkat"].map(test["id_perangkat"].value_counts())
test["kategori_id_perangkat"] = np.where(test['freq_id_perangkat'] > 1, '>1', 'once')

test["freq_country"] = test["country"].map(test["country"].value_counts())
test["kategori_country"] = np.where(test['freq_country'] > 1000, '>1000', '<=1000')

train.head()

,id_pengguna,waktu_pendaftaran_akun,waktu_pembelian,total_harga_pembelian,id_perangkat,sumber,browser,gender,umur,alamat_IP,fraud,country,day_difference,sumber_Langsung_or_SEO,browser_c_or_ie,freq_id_perangkat,kategori_id_perangkat,freq_country,kategori_country
0,286873,2015-06-07 12:58:00,2015-09-27 16:32:00,14,DNTIEECGGDXHF,Iklan,Opera,F,38,241720722,0,China,112,0,0,1,once,6062,>1000
1,244392,2015-05-01 15:29:00,2015-08-27 14:21:00,11,MECRFNJGDPGOB,SEO,Chrome,M,36,4172794236,0,Unknown,117,1,1,1,once,10039,>1000
2,20720,2015-03-29 18:04:00,2015-06-13 06:50:00,52,WLTSYVLLASJOZ,SEO,Internet Explorer,M,26,1113432879,0,United States,75,1,1,1,once,29432,>1000
3,198755,2015-01-29 00:49:00,2015-02-17 21:49:00,13,TWGNWOWURZFMY,SEO,Safari,F,38,1339308896,0,Germany,19,1,0,1,once,1820,>1000
4,64996,2015-01-17 19:30:00,2015-03-01 09:17:00,23,XMQENRCEILYAJ,Langsung,Safari,F,19,460238214,0,Philippines,42,1,0,1,once,84,<=1000


In [18]:
train_id_n_country = train[["kategori_id_perangkat", "kategori_country"]]

id_n_country_encoded_arr = one_hot_encoder.fit_transform(train_id_n_country)
id_n_country_encoded_df = pd.DataFrame(id_n_country_encoded_arr, columns=one_hot_encoder.get_feature_names_out(train_id_n_country.columns))
id_n_country_encoded_df.head()

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,kategori_id_perangkat_>1,kategori_id_perangkat_once,kategori_country_<=1000,kategori_country_>1000
0,0.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0
4,0.0,1.0,1.0,0.0


In [19]:
id_n_country_encoded_df["fraud"] = train["fraud"]

In [20]:
id_n_country_corr_matrix = id_n_country_encoded_df.corr()
id_n_country_corr_matrix["fraud"].sort_values(ascending=False)

fraud                         1.000000
kategori_id_perangkat_>1      0.576351
kategori_country_<=1000       0.003382
kategori_country_>1000       -0.003382
kategori_id_perangkat_once   -0.576351
Name: fraud, dtype: float64

In [21]:
train["freq_id_perangkat_>1"] = np.where(train['freq_id_perangkat'] > 1, 1, 0)
test["freq_id_perangkat_>1"] = np.where(test['freq_id_perangkat'] > 1, 1, 0)
train.head()

,id_pengguna,waktu_pendaftaran_akun,waktu_pembelian,total_harga_pembelian,id_perangkat,sumber,browser,gender,umur,alamat_IP,fraud,country,day_difference,sumber_Langsung_or_SEO,browser_c_or_ie,freq_id_perangkat,kategori_id_perangkat,freq_country,kategori_country,freq_id_perangkat_>1
0,286873,2015-06-07 12:58:00,2015-09-27 16:32:00,14,DNTIEECGGDXHF,Iklan,Opera,F,38,241720722,0,China,112,0,0,1,once,6062,>1000,0
1,244392,2015-05-01 15:29:00,2015-08-27 14:21:00,11,MECRFNJGDPGOB,SEO,Chrome,M,36,4172794236,0,Unknown,117,1,1,1,once,10039,>1000,0
2,20720,2015-03-29 18:04:00,2015-06-13 06:50:00,52,WLTSYVLLASJOZ,SEO,Internet Explorer,M,26,1113432879,0,United States,75,1,1,1,once,29432,>1000,0
3,198755,2015-01-29 00:49:00,2015-02-17 21:49:00,13,TWGNWOWURZFMY,SEO,Safari,F,38,1339308896,0,Germany,19,1,0,1,once,1820,>1000,0
4,64996,2015-01-17 19:30:00,2015-03-01 09:17:00,23,XMQENRCEILYAJ,Langsung,Safari,F,19,460238214,0,Philippines,42,1,0,1,once,84,<=1000,0


In [22]:
train["id_perangkat"].nunique()

71299

In [23]:
train["jam_pembelian"] = train["waktu_pembelian"].dt.hour
train.head()
train["jam_pembelian"].corr(train["fraud"])

0.0002934632217354246

In [24]:
used_columns = ["id_pengguna","total_harga_pembelian", "umur", "day_difference", "sumber_Langsung_or_SEO", "browser_c_or_ie", "freq_id_perangkat_>1", "fraud"]
train_final = train[used_columns]

test_final = test[["id_pengguna","total_harga_pembelian", "umur", "day_difference", "sumber_Langsung_or_SEO", "browser_c_or_ie", "freq_id_perangkat_>1"]]
train_final.head()

,id_pengguna,total_harga_pembelian,umur,day_difference,sumber_Langsung_or_SEO,browser_c_or_ie,freq_id_perangkat_>1,fraud
0,286873,14,38,112,0,0,0,0
1,244392,11,36,117,1,1,0,0
2,20720,52,26,75,1,1,0,0
3,198755,13,38,19,1,0,0,0
4,64996,23,19,42,1,0,0,0


MODEL (VALIDATION)

In [25]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn. import Ridge
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import QuantileTransformer

In [26]:
#features = ["total_harga_pembelian", "umur", "day_difference", "sumber_Langsung_or_SEO", "browser_c_or_ie", "freq_id_perangkat_>1"]
#user_id = train_final["id_pengguna"]
#X = train_final[features]
#y = train_final["fraud"]

In [27]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
#smote = SMOTE(random_state=42)
#X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
#X_train_qt = QuantileTransformer(n_quantiles=100).fit_transform(X_train_sm)
#X_test_qt  = QuantileTransformer(n_quantiles=100).fit_transform(X_test)

In [28]:
#lr = LogisticRegression(random_state = 42)
#lr.fit(X_train, y_train)

#y_pred = lr.predict(X_test_qt)
#print("Balanced Accuracy Score:")
#print(balanced_accuracy_score(y_test, y_pred))

In [29]:
#from sklearn.ensemble import VotingClassifier
#rf = RandomForestClassifier(random_state=42)
#lr = LogisticRegression(random_state = 42)
#voting_clf = VotingClassifier(estimators=[('lr', rf), ('dt', lr)], voting='hard')
#voting_clf.fit(X_train_sm, y_train_sm)

# Prediksi dengan voting classifier

#y_pred = voting_clf.predict(X_test_qt)

#print("Balanced Accuracy Score:")
#print(balanced_accuracy_score(y_test, y_pred))

In [30]:
# Feature Importance
#feature_importances = rf.feature_importances_
#feature_names = [f'Feature {i}' for i in range(X.shape[1])]
#importance_df = pd.DataFrame({
#    'Feature': feature_names,    'Importance': feature_importances})

In [31]:
#importance_df

In [32]:
# models = {
#     'Logistic Regression': LogisticRegression(random_state=42),
#     'Random Forest': RandomForestClassifier(random_state=42),
#     'voting_clf' : VotingClassifier(estimators=[('lr', RandomForestClassifier(random_state=42)), ('dt', LogisticRegression(random_state=42))], voting='hard'),
#     'voting_clf2' : VotingClassifier(estimators=[('lr', RandomForestClassifier(random_state=42)), ('dt', LogisticRegression(random_state=42))], voting='soft')
# }

#for name, model in models.items():
#    model.fit(X_train_sm, y_train_sm)
#    y_pred = model.predict(X_test_qt)
    
#    print(f"{name} Classification Report:")
#    print(classification_report(y_test, y_pred))
#    print(f"{name} Balanced Accuracy Score:")
#    print(balanced_accuracy_score(y_test, y_pred))
#    print("\n")

MODEL W/ TEST DATA

In [33]:
features = ["total_harga_pembelian", "umur", "day_difference", "sumber_Langsung_or_SEO", "browser_c_or_ie", "freq_id_perangkat_>1"]
X_train = train_final[features]
y_train = train_final["fraud"]

test_user_id = test_final["id_pengguna"]
X_test = test_final[features]

In [34]:
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
X_train_qt = QuantileTransformer(n_quantiles=100).fit_transform(X_train_sm)
X_test_qt  = QuantileTransformer(n_quantiles=100).fit_transform(X_test)

In [35]:
from sklearn.ensemble import VotingClassifier
voting_clf2 = VotingClassifier(estimators=[('lr', RandomForestClassifier(random_state=42)), ('dt', LogisticRegression(random_state=42)), ('rc', Ridgeclass)], voting='soft')
voting_clf2.fit(X_train_sm, y_train_sm)
y_pred = voting_clf2.predict(X_test_qt)

predictions = pd.DataFrame(y_pred, columns=["fraud"])
predictions

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


,fraud
0,0
1,0
2,0
3,0
4,0
...,...
75551,0
75552,0
75553,0
75554,0


In [36]:
submission = pd.DataFrame(test_user_id, columns=["id_pengguna"])
submission["fraud"] = predictions
submission

,id_pengguna,fraud
0,28006,0
1,245189,0
2,58345,0
3,218574,0
4,49779,0
...,...,...
75551,18893,0
75552,354167,0
75553,139213,0
75554,130538,0


In [37]:
submission.to_csv("submisi_hp.csv", index=False)